In [1]:
#needed after restructuring of directory
import os
os.chdir('/Users/alexchandler/UFC_Prediction_2022')
os.getcwd()

'/Users/alexchandler/UFC_Prediction_2022'

In [1]:
#This entire notebook shouldn't take more than 2 hours to run
#run this notebook after running 
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as npy
import re
from datetime import datetime, timedelta, date
import random

In [2]:
#updated scraped fight data (after running fight_hist_updated function from UFC_data_scraping file)
fight_hist=pd.read_csv('src/content/data/processed/fight_hist.csv',sep=',')
#all stats fight history file which is one update behind fight_hist
ufcfightscrap=pd.read_csv('src/content/data/processed/ufc_fights_crap.csv',sep=',',low_memory=False)
#updated scraped fighter data (after running fight_hist_updated function from UFC_data_scraping file)
ufcfighterscrap=pd.read_csv('src/content/data/processed/fighter_stats.csv',sep=',')
#most recent fight in fight_hist versus most recent fight in ufcfightscrap

In [3]:
#this cell contains all functions defined for building columns in ufcfightscrap
#converts from '%B %d, %Y' (i.e. August 22, 2020) to date (i.e. 2020-08-22)
def convert_to_datetime(day1):
    return datetime.strptime(day1, '%B %d, %Y').date()
convert_to_datetime_vect= npy.vectorize(convert_to_datetime)

#this age function is written in such a stupid way
def age(birthDate,day=date.today(),form1='%B %d, %Y',form2='%B %d, %Y'):
    if birthDate=='--':
        aa='unknown'
    elif type(birthDate)==str and not type(day)==str:
        bd=datetime.strptime(birthDate, form1)
        today = day
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    elif type(birthDate)==str and type(day)==str:
        bd=datetime.strptime(birthDate, form1)
        today = datetime.strptime(day, form2)
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    elif not type(birthDate)==str and type(day)==str:
        bd=birthDate
        today = datetime.strptime(day, form2)
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    else:
        bd=birthDate
        today = day
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    return float(aa)

age_vect= npy.vectorize(age)

#zero function (needed for reversals)
def zero(x):
    return 0

zero_vect= npy.vectorize(zero)

def fighter_age(fighter,day=date.today(),form1='%B %d, %Y',form2='%B %d, %Y'):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            dob=datetime.strptime(ufcfighterscrap['dob'][i], '%b %d, %Y').strftime('%B %d, %Y')
            a=age(dob,day,form1,form2)
            break
    return float(a)

fighter_age_vect= npy.vectorize(fighter_age)

def wins_before(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
    return float(summ)

def losses_before(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
    return float(summ)

#functions for height and reach
def fighter_height(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['height'][i]
            break
    if a=='--' or type(a)==int:
        b='unknown'
    elif a[4]=='"':
        b=int(a[0])*30.48+int(a[3])*2.54
    else:
        b=int(a[0])*30.48+int(a[3]+a[4])*2.54
    try:
        return float(b)
    except:
        return b

def fighter_reach(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['reach'][i]
            break
    if a=='--' or type(a)==int:
        b='unknown'
    else:
        b=int(a[0]+a[1])*2.54
    try:
        return float(b)
    except:
        return b

wins_before_vect= npy.vectorize(wins_before)
losses_before_vect= npy.vectorize(losses_before)
fighter_height_vect= npy.vectorize(fighter_height)
fighter_reach_vect= npy.vectorize(fighter_reach)

#day1 should be input in the form '%B %d, %Y' i.e. 'August 20, 1962'
#conversions can be made via day=datetime.strptime(ufcfightsML_known_df['date'][i], '%B %d, %Y').strftime('%b %d, %Y')
def time_diff(day1,day2=date.today()):
    if day2==date.today():
        answer=(day2-datetime.strptime(day, '%B %d, %Y')).days
    else:
        answer=(datetime.strptime(day2, '%B %d, %Y')-datetime.strptime(day1, '%B %d, %Y')).days
    return answer

#we now vectorize this to use in pandas/numpy
time_diff_vect= npy.vectorize(time_diff)

#can make a single function to do all of these... actually maybe the count function would even work as is

def L5Y_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L5Y_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L2Y_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def L2Y_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1) and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
    return float(summ)

def ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1) and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
    return float(summ)

def L5Y_ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L5Y_ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L2Y_ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def L2Y_ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1) and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
    return float(summ)

def sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1) and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
    return float(summ)

def L5Y_sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L5Y_sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L2Y_sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def L2Y_sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

#vectorize all of these functions
L5Y_wins_vect= npy.vectorize(L5Y_wins)
L5Y_losses_vect= npy.vectorize(L5Y_losses)
L2Y_wins_vect= npy.vectorize(L2Y_wins)
L2Y_losses_vect= npy.vectorize(L2Y_losses)
ko_wins_vect= npy.vectorize(ko_wins)
ko_losses_vect= npy.vectorize(ko_losses)
L5Y_ko_wins_vect= npy.vectorize(L5Y_ko_wins)
L5Y_ko_losses_vect= npy.vectorize(L5Y_ko_losses)
L2Y_ko_wins_vect= npy.vectorize(L2Y_ko_wins)
L2Y_ko_losses_vect= npy.vectorize(L2Y_ko_losses)
sub_wins_vect= npy.vectorize(sub_wins)
sub_losses_vect= npy.vectorize(sub_losses)
L5Y_sub_wins_vect= npy.vectorize(L5Y_sub_wins)
L5Y_sub_losses_vect= npy.vectorize(L5Y_sub_losses)
L2Y_sub_wins_vect= npy.vectorize(L2Y_sub_wins)
L2Y_sub_losses_vect= npy.vectorize(L2Y_sub_losses)

#for columns like fighter_rec which contains the information for the opponent as well, we use the following
def opponent_column(stat):
    col=dict()
    for i in range(len(ufcfightscrap['fighter'])):
        if i%2==0:
            col[i]=ufcfightscrap[stat][i+1]
        else:
            col[i]=ufcfightscrap[stat][i-1]
    statdict={'stat':col}
    return pd.DataFrame (statdict, columns = ['stat'])

#enter date unabbreviated 'July 4, 2019'
#here the average gives avg per fight. Later in avg_count we change to average per time spent in octagon
def count(stat, guy,inf_abs, total_L5Y_L2Y_avg, day1=date.today().strftime('%B %d, %Y')):
    summ=0
    if total_L5Y_L2Y_avg=='total' or total_L5Y_L2Y_avg=='avg':
        good_indices_1=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date'][i],day1)>0]
    elif total_L5Y_L2Y_avg=='L2Y':
        good_indices_1=[i for i in ufcfightscrap.index.values if 0<time_diff(ufcfightscrap['date'][i],day1)<730]
    else:
        good_indices_1=[i for i in ufcfightscrap.index.values if 0<time_diff(ufcfightscrap['date'][i],day1)<1825]
    if inf_abs=='inf':
        good_indices_2=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    else:
        good_indices_2=[i for i in ufcfightscrap.index.values if ufcfightscrap['opponent'][i]==guy]
    good_indices=[i for i in good_indices_1 if i in good_indices_2]
    if total_L5Y_L2Y_avg!='avg':
        for i in good_indices:
            summ+=ufcfightscrap[stat][i]
    else:
        for i in good_indices:
            summ+=ufcfightscrap[stat][i]
        day1=convert_date_to_abbrev(day1)
        number_fights=wins_before(guy,day1)+losses_before(guy,day1)
        summ=summ/float(number_fights)
    return summ
        
#note a better average is per time not per fight. We will do each stat as an average per one minutes

#enter time in the form 'August 24, 2018'
def time_in_octagon(guy,day1=date.today().strftime('%B %d, %Y')):
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['fighter'][i]==guy] 
    for i in good_indices:
        if ufcfightscrap['time'][i][2]==':':
            summ=int(ufcfightscrap['time'][i][0:2])+int(ufcfightscrap['time'][i][3:])/60.0
        else:
            summ+=5*(ufcfightscrap['round'][i]-1)+int(ufcfightscrap['time'][i][0])+int(ufcfightscrap['time'][i][2:])/60.0
    return summ

#enter date unabbreviated 'July 4, 2019'
#gives takedowns per minute
def avg_count(stat, guy,inf_abs, day1=date.today().strftime('%B %d, %Y')):
    summ=0
    if inf_abs=='inf':
        good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy and time_diff(ufcfightscrap['date'][i],day1)>0]
    else:
        good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['opponent'][i]==guy and time_diff(ufcfightscrap['date'][i],day1)>0]
    for i in good_indices:
        summ+=ufcfightscrap[stat][i]
    t= time_in_octagon(guy,day1)
    if t==0:
        summ=0
    else:
        summ=summ/t
    return summ

#vectorize these functions
count_vect= npy.vectorize(count)
avg_count_vect= npy.vectorize(avg_count)
    


def stance(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['stance'][i]
            break
    if a=='Orthodox':
        return 0
    elif a=='Switch':
        return 1
    elif a=='Southpaw':
        return 2
    elif a=='Open Stance':
        return 3
    elif a=='Sideways':
        return 4
    else:
        return 5
    
stance_vect= npy.vectorize(stance)

#turning csv files to json files
import csv
import json
 
 
# Function to convert a CSV to JSON
# Takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath, column):
     
    # create a dictionary
    data = {}
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:
             
            # primary key given by column variable
            key = rows[column]
            data[key] = rows
 
    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
        
ufc_fights_graph = pd.read_csv('src/content/data/processed/ufc_fights_crap.csv',low_memory=False)
odd_indices = range(1,len(ufc_fights_graph.index),2)
ufc_fights_graph = ufc_fights_graph.drop(odd_indices)
ufc_fights_graph = ufc_fights_graph[['fighter', 'opponent', 'method', 'date', 'division']]
ufc_fights_graph = ufc_fights_graph.reset_index(drop=True);
ufc_wins_list=[]

for i in ufc_fights_graph.index:
    temp_list = []
    temp_list.append(ufc_fights_graph['fighter'][i])
    temp_list.append(ufc_fights_graph['opponent'][i])
    temp_list.append(ufc_fights_graph['date'][i])
    temp_list.append(ufc_fights_graph['division'][i])
    ufc_wins_list.append(temp_list)
    
def fight_math(fighter,opponent,date,years):
    fighter_advantage = 0
    ufc_wins_list_l5y = [fight for fight in ufc_wins_list if 0<time_diff(fight[2],date)<years*365]
    fighter_wins = [fight[1] for fight in ufc_wins_list_l5y if fight[0]==fighter]
    fighter_wins.append(fighter)
    fighter_wins_wins = [fight[1] for fight in ufc_wins_list_l5y if fight[0] in fighter_wins]
    relevant_wins = list(set(fighter_wins+fighter_wins_wins))
    fight_math_wins = [fight for fight in ufc_wins_list_l5y if (fight[0] in relevant_wins and fight[1]==opponent)]
    fighter_advantage+=len(fight_math_wins)
    return fighter_advantage

def fight_math_diff(fighter,opponent,date,years):
    return fight_math(fighter,opponent,date,years)-fight_math(opponent,fighter,date,years)
fight_math_diff_vect = npy.vectorize(fight_math_diff)

#need to define variable "years" before calling this function
#perhaps a better score would weight more recent fights more strongly (weight drops by 1/3 every year?)
def fighter_score(fighter, date, years):
    fighter_score = 0
    ufc_wins_list_l5y = [fight for fight in ufc_wins_list if 0<time_diff(fight[2],date)<years*365]
    #calculating contribution from wins
    fighter_wins = [fight[1] for fight in ufc_wins_list_l5y if fight[0]==fighter]
    fighter_wins_wins = [fight[1] for fight in ufc_wins_list_l5y if fight[0] in fighter_wins]
    relevant_wins = list(set(fighter_wins+fighter_wins_wins))
    #calculating contribution from losses
    fighter_losses = [fight[0] for fight in ufc_wins_list_l5y if fight[1]==fighter]
    fighter_losses_losses = [fight[0] for fight in ufc_wins_list_l5y if fight[1] in fighter_wins]
    relevant_losses = list(set(fighter_losses+fighter_losses_losses))
    return len(relevant_wins)-len(relevant_losses)

def fighter_score_diff(fighter,opponent,date, years):
    return fighter_score(fighter,date,years)-fighter_score(opponent,date,years)

fighter_score_diff_vect = npy.vectorize(fighter_score_diff)

         

In [4]:
update_time=time_diff(ufcfightscrap['date'][0],fight_hist['date'][0])
print(update_time)

0


In [5]:
#this gives the new rows in fight_hist which do not appear in ufcfightscrapd
new_rows=fight_hist.loc[time_diff_vect(fight_hist['date'],fight_hist['date'][0])<update_time]
#convert to numpy array
numpy_new_rows=new_rows.values
#gives new_rows as a dataframe instead of a clone
test_new_rows=pd.DataFrame(data=numpy_new_rows,    # 1st column as index
                columns=new_rows.columns)  # 1st row as the column names

In [6]:
#adding a datetime column for test_new_rows
#test_new_rows['datetime']=convert_to_datetime_vect(test_new_rows['date'])
#setting up columns for rec,wins,losses,ties (can be done in one line each as we do later with other stats)
#run time for each is 20 min per pair
test_new_rows['fighter_wins']=wins_before_vect(test_new_rows['fighter'],test_new_rows['date'])
#column for fighter losses in test_new_rows
test_new_rows['fighter_losses']=losses_before_vect(test_new_rows['fighter'],test_new_rows['date'])
#column for fighter age in test_new_rows
test_new_rows['fighter_age']=fighter_age_vect(test_new_rows['fighter'],test_new_rows['date'])
#column for fighter height in test_new_rows
test_new_rows['fighter_height']=test_new_rows['fighter'].apply(fighter_height)
#column for fighter reach in test_new_rows
test_new_rows['fighter_reach']=test_new_rows['fighter'].apply(fighter_height)

ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set

In [ ]:
test_new_rows

In [7]:
#making columns for fighter_L5Y_wins losses etc (about 5 minutes or less)
test_new_rows['fighter_L5Y_wins']=L5Y_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L5Y_losses']=L5Y_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L2Y_wins']=L2Y_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L2Y_losses']=L2Y_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_ko_wins']=ko_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_ko_losses']=ko_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L5Y_ko_wins']=L5Y_ko_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L5Y_ko_losses']=L5Y_ko_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L2Y_ko_wins']=L2Y_ko_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L2Y_ko_losses']=L2Y_ko_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_sub_wins']=sub_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_sub_losses']=sub_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L5Y_sub_wins']=L5Y_sub_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L5Y_sub_losses']=L5Y_sub_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L2Y_sub_wins']=L2Y_sub_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L2Y_sub_losses']=L2Y_sub_losses_vect(test_new_rows['fighter'],test_new_rows['date'])

ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set

In [8]:
time test_row = avg_count_vect('knockdowns',test_new_rows['fighter'],'inf',test_new_rows['date'])


CPU times: user 4.75 s, sys: 15.9 ms, total: 4.76 s
Wall time: 4.77 s


In [9]:
test_new_rows['fighter_inf_knockdowns_avg']=test_row;
test_new_rows

date                                          fight_url  \
0   March 26, 2022  http://ufcstats.com/fight-details/c4e866d9dd60...   
1   March 26, 2022  http://ufcstats.com/fight-details/c4e866d9dd60...   
2   March 26, 2022  http://ufcstats.com/fight-details/1b650c76387c...   
3   March 26, 2022  http://ufcstats.com/fight-details/1b650c76387c...   
4   March 26, 2022  http://ufcstats.com/fight-details/f3c37e23ca30...   
5   March 26, 2022  http://ufcstats.com/fight-details/f3c37e23ca30...   
6   March 26, 2022  http://ufcstats.com/fight-details/410e65e4d178...   
7   March 26, 2022  http://ufcstats.com/fight-details/410e65e4d178...   
8   March 26, 2022  http://ufcstats.com/fight-details/3f92025b29e0...   
9   March 26, 2022  http://ufcstats.com/fight-details/3f92025b29e0...   
10  March 26, 2022  http://ufcstats.com/fight-details/f6afaf2a39fa...   
11  March 26, 2022  http://ufcstats.com/fight-details/f6afaf2a39fa...   
12  March 26, 2022  http://ufcstats.com/fight-details/bb31c77bddd1...   
13  March 26, 2022  http://ufcstats.com/fight-details/bb31c77bddd1...   
14  March 26, 2022  http://ufcstats.com/fight-details/d0b7a79d3e7f...   
15  March 26, 2022  http://ufcstats.com/fight-details/d0b7a79d3e7f...   
16  March 26, 2022  http://ufcstats.com/fight-details/6ed495c177d6...   
17  March 26, 2022  http://ufcstats.com/fight-details/6ed495c177d6...   
18  March 26, 2022  http://ufcstats.com/fight-details/3ecccdab76ba...   
19  March 26, 2022  http://ufcstats.com/fight-details/3ecccdab76ba...   
20  March 26, 2022  http://ufcstats.com/fight-details/82b2b97b85c7...   
21  March 26, 2022  http://ufcstats.com/fight-details/82b2b97b85c7...   
22  March 26, 2022  http://ufcstats.com/fight-details/97e522591ac7...   
23  March 26, 2022  http://ufcstats.com/fight-details/97e522591ac7...   

                                            event_url result  \
0   http://ufcstats.com/event-details/1fac46d466ab...      W   
1   http://ufcstats.com/event-details/1fac46d466ab...      L   
2   http://ufcstats.com/event-details/1fac46d466ab...      W   
3   http://ufcstats.com/event-details/1fac46d466ab...      L   
4   http://ufcstats.com/event-details/1fac46d466ab...      W   
5   http://ufcstats.com/event-details/1fac46d466ab...      L   
6   http://ufcstats.com/event-details/1fac46d466ab...      W   
7   http://ufcstats.com/event-details/1fac46d466ab...      L   
8   http://ufcstats.com/event-details/1fac46d466ab...      W   
9   http://ufcstats.com/event-details/1fac46d466ab...      L   
10  http://ufcstats.com/event-details/1fac46d466ab...      W   
11  http://ufcstats.com/event-details/1fac46d466ab...      L   
12  http://ufcstats.com/event-details/1fac46d466ab...      W   
13  http://ufcstats.com/event-details/1fac46d466ab...      L   
14  http://ufcstats.com/event-details/1fac46d466ab...      W   
15  http://ufcstats.com/event-details/1fac46d466ab...      L   
16  http://ufcstats.com/event-details/1fac46d466ab...      W   
17  http://ufcstats.com/event-details/1fac46d466ab...      L   
18  http://ufcstats.com/event-details/1fac46d466ab...      W   
19  http://ufcstats.com/event-details/1fac46d466ab...      L   
20  http://ufcstats.com/event-details/1fac46d466ab...      W   
21  http://ufcstats.com/event-details/1fac46d466ab...      L   
22  http://ufcstats.com/event-details/1fac46d466ab...      W   
23  http://ufcstats.com/event-details/1fac46d466ab...      L   

                 fighter              opponent              division  method  \
0         Curtis Blaydes         Chris Daukaus           Heavyweight  KO/TKO   
1          Chris Daukaus        Curtis Blaydes           Heavyweight  KO/TKO   
2           Alexa Grasso           Joanne Wood     Women's Flyweight     SUB   
3            Joanne Wood          Alexa Grasso     Women's Flyweight     SUB   
4        Bryan Barberena            Matt Brown          Welterweight   S-DEC   
5             Matt Brown       Bryan Barberena          Welterweight   S-DEC   
6        Kai Kara-France   

In [10]:
#forming columns for fighter_inf_knockdowns_avg etc... for (1500 new examples takes 90 minutes)
test_new_rows['fighter_inf_knockdowns_avg']=avg_count_vect('knockdowns',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_pass_avg']=avg_count_vect('pass',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_reversals_avg']=zero_vect(test_new_rows['fighter'])
test_new_rows['fighter_inf_sub_attempts_avg']=avg_count_vect('sub_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_takedowns_landed_avg']=avg_count_vect('takedowns_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_takedowns_attempts_avg']=avg_count_vect('takedowns_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
print('quarter done')
test_new_rows['fighter_inf_sig_strikes_landed_avg']=avg_count_vect('sig_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_sig_strikes_attempts_avg']=avg_count_vect('sig_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_total_strikes_landed_avg']=avg_count_vect('total_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_total_strikes_attempts_avg']=avg_count_vect('total_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_head_strikes_landed_avg']=avg_count_vect('head_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_head_strikes_attempts_avg']=avg_count_vect('head_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
print('half done')
test_new_rows['fighter_inf_body_strikes_landed_avg']=avg_count_vect('body_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_leg_strikes_attempts_avg']=avg_count_vect('leg_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_distance_strikes_landed_avg']=avg_count_vect('distance_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
print('almost done')
test_new_rows['fighter_inf_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_clinch_strikes_attempts_avg']=avg_count_vect('clinch_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_ground_strikes_landed_avg']=avg_count_vect('ground_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_ground_strikes_attempts_avg']=avg_count_vect('ground_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])


quarter done
half done
almost done


In [11]:
#forming columns for fighter_abs_knockdowns_avg etc...
test_new_rows['fighter_abs_knockdowns_avg']=avg_count_vect('knockdowns',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_pass_avg']=avg_count_vect('pass',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_reversals_avg']=zero_vect(test_new_rows['fighter'])
test_new_rows['fighter_abs_sub_attempts_avg']=avg_count_vect('sub_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_takedowns_landed_avg']=avg_count_vect('takedowns_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_takedowns_attempts_avg']=avg_count_vect('takedowns_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
print('quarter done')
test_new_rows['fighter_abs_sig_strikes_landed_avg']=avg_count_vect('sig_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_sig_strikes_attempts_avg']=avg_count_vect('sig_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_total_strikes_landed_avg']=avg_count_vect('total_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_total_strikes_attempts_avg']=avg_count_vect('total_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_head_strikes_landed_avg']=avg_count_vect('head_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_head_strikes_attempts_avg']=avg_count_vect('head_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
print('half done')
test_new_rows['fighter_abs_body_strikes_landed_avg']=avg_count_vect('body_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_leg_strikes_attempts_avg']=avg_count_vect('leg_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_distance_strikes_landed_avg']=avg_count_vect('distance_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
print('almost done')
test_new_rows['fighter_abs_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_clinch_strikes_attempts_avg']=avg_count_vect('clinch_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_ground_strikes_landed_avg']=avg_count_vect('ground_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_ground_strikes_attempts_avg']=avg_count_vect('ground_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])

quarter done
half done
almost done


In [12]:
#opponent columns for wins etc
test_new_rows['opponent_wins']=opponent_column('fighter_wins')
test_new_rows['opponent_losses']=opponent_column('fighter_losses')
test_new_rows['opponent_age']=opponent_column('fighter_age')
test_new_rows['opponent_height']=opponent_column('fighter_height')
test_new_rows['opponent_reach']=opponent_column('fighter_reach')
#making columns for opponent_L5Y_wins losses etc
print('quarter done')
test_new_rows['opponent_L5Y_wins']=opponent_column('fighter_L5Y_wins')
test_new_rows['opponent_L5Y_losses']=opponent_column('fighter_L5Y_losses')
test_new_rows['opponent_L2Y_wins']=opponent_column('fighter_L2Y_wins')
test_new_rows['opponent_L2Y_losses']=opponent_column('fighter_L2Y_losses')
test_new_rows['opponent_ko_wins']=opponent_column('fighter_ko_wins')
test_new_rows['opponent_ko_losses']=opponent_column('fighter_ko_losses')
print('half done')
test_new_rows['opponent_L5Y_ko_wins']=opponent_column('fighter_L5Y_ko_wins')
test_new_rows['opponent_L5Y_ko_losses']=opponent_column('fighter_L5Y_ko_losses')
test_new_rows['opponent_L2Y_ko_wins']=opponent_column('fighter_L2Y_ko_wins')
test_new_rows['opponent_L2Y_ko_losses']=opponent_column('fighter_L2Y_ko_losses')
test_new_rows['opponent_sub_wins']=opponent_column('fighter_sub_wins')
print('almost done')
test_new_rows['opponent_sub_losses']=opponent_column('fighter_sub_losses')
test_new_rows['opponent_L5Y_sub_wins']=opponent_column('fighter_L5Y_sub_wins')
test_new_rows['opponent_L5Y_sub_losses']=opponent_column('fighter_L5Y_sub_losses')
test_new_rows['opponent_L2Y_sub_wins']=opponent_column('fighter_L2Y_sub_wins')
test_new_rows['opponent_L2Y_sub_losses']=opponent_column('fighter_L2Y_sub_losses')

quarter done
half done
almost done


In [13]:
#forming columns for opponent_inf_knockdowns_avg etc...
test_new_rows['opponent_inf_knockdowns_avg']=avg_count_vect('knockdowns',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_pass_avg']=avg_count_vect('pass',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_reversals_avg']=zero_vect(test_new_rows['opponent'])
test_new_rows['opponent_inf_sub_attempts_avg']=avg_count_vect('sub_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_takedowns_landed_avg']=avg_count_vect('takedowns_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_takedowns_attempts_avg']=avg_count_vect('takedowns_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
print('quarter done')
test_new_rows['opponent_inf_sig_strikes_landed_avg']=avg_count_vect('sig_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_sig_strikes_attempts_avg']=avg_count_vect('sig_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_total_strikes_landed_avg']=avg_count_vect('total_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_total_strikes_attempts_avg']=avg_count_vect('total_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_head_strikes_landed_avg']=avg_count_vect('head_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
print('half done')
test_new_rows['opponent_inf_head_strikes_attempts_avg']=avg_count_vect('head_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_body_strikes_landed_avg']=avg_count_vect('body_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_leg_strikes_attempts_avg']=avg_count_vect('leg_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_distance_strikes_landed_avg']=avg_count_vect('distance_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
print('almost done')
test_new_rows['opponent_inf_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_clinch_strikes_attempts_avg']=avg_count_vect('clinch_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_ground_strikes_landed_avg']=avg_count_vect('ground_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_ground_strikes_attempts_avg']=avg_count_vect('ground_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])

quarter done
half done


/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_24579/2569836320.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_inf_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_24579/2569836320.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_inf_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',test_new_rows['opponent'],'inf',test_

almost done


/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_24579/2569836320.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_inf_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_24579/2569836320.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_inf_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',test_new_rows['opponent

In [14]:
#forming columns for opponent_abs_knockdowns_avg etc...
test_new_rows['opponent_abs_knockdowns_avg']=avg_count_vect('knockdowns',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_pass_avg']=avg_count_vect('pass',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_reversals_avg']=zero_vect(test_new_rows['opponent'])
test_new_rows['opponent_abs_sub_attempts_avg']=avg_count_vect('sub_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_takedowns_landed_avg']=avg_count_vect('takedowns_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_takedowns_attempts_avg']=avg_count_vect('takedowns_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_sig_strikes_landed_avg']=avg_count_vect('sig_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
print('quarter done')
test_new_rows['opponent_abs_sig_strikes_attempts_avg']=avg_count_vect('sig_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_total_strikes_landed_avg']=avg_count_vect('total_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_total_strikes_attempts_avg']=avg_count_vect('total_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_head_strikes_landed_avg']=avg_count_vect('head_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_head_strikes_attempts_avg']=avg_count_vect('head_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_body_strikes_landed_avg']=avg_count_vect('body_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
print('half done')
test_new_rows['opponent_abs_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_leg_strikes_attempts_avg']=avg_count_vect('leg_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_distance_strikes_landed_avg']=avg_count_vect('distance_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
print('almost done')
test_new_rows['opponent_abs_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_clinch_strikes_attempts_avg']=avg_count_vect('clinch_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_ground_strikes_landed_avg']=avg_count_vect('ground_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_ground_strikes_attempts_avg']=avg_count_vect('ground_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])

/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_24579/290036775.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_abs_knockdowns_avg']=avg_count_vect('knockdowns',test_new_rows['opponent'],'abs',test_new_rows['date'])
/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_24579/290036775.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_abs_pass_avg']=avg_count_vect('pass',test_new_rows['opponent'],'abs',test_new_rows['date'])
/var/folders/zr/jttn57m976g1m1_y2p2t

quarter done


/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_24579/290036775.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_abs_sig_strikes_attempts_avg']=avg_count_vect('sig_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_24579/290036775.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_abs_total_strikes_landed_avg']=avg_count_vect('total_strikes_landed',test_new_rows['opponent'],'abs',test_

half done


/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_24579/290036775.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_abs_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_24579/290036775.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_abs_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',test_new_rows['opponent'],'abs',test_ne

almost done


/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_24579/290036775.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_abs_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_24579/290036775.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_abs_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',test_new_rows['opponent']

In [15]:
test_new_rows['fighter_stance']=stance_vect(test_new_rows['fighter'])
test_new_rows['opponent_stance']=stance_vect(test_new_rows['opponent'])

/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_24579/86544192.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['fighter_stance']=stance_vect(test_new_rows['fighter'])
/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_24579/86544192.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_new_rows['opponent_stance']=stance_vect(test_new_rows['opponent'])


In [23]:
test_new_rows.columns

Index(['date', 'fight_url', 'event_url', 'result', 'fighter', 'opponent',
       'division', 'method', 'round', 'time',
       ...
       'opponent_abs_leg_strikes_landed_avg',
       'opponent_abs_leg_strikes_attempts_avg',
       'opponent_abs_distance_strikes_landed_avg',
       'opponent_abs_distance_strikes_attempts_avg',
       'opponent_abs_clinch_strikes_landed_avg',
       'opponent_abs_clinch_strikes_attempts_avg',
       'opponent_abs_ground_strikes_landed_avg',
       'opponent_abs_ground_strikes_attempts_avg', 'fighter_stance',
       'opponent_stance'],
      dtype='object', length=167)

In [27]:
#adding new columns for fight math and exporting to csvs
test_new_rows['1-fight_math'] = fight_math_diff_vect(test_new_rows['fighter'], test_new_rows['opponent'], test_new_rows['date'],1)
test_new_rows['6-fight_math'] = fight_math_diff_vect(test_new_rows['fighter'], test_new_rows['opponent'], test_new_rows['date'],6)
test_new_rows['4-fighter_score_diff'] = fighter_score_diff_vect(test_new_rows['fighter'], test_new_rows['opponent'], test_new_rows['date'],4)
test_new_rows['9-fighter_score_diff'] = fighter_score_diff_vect(test_new_rows['fighter'], test_new_rows['opponent'], test_new_rows['date'],9)
test_new_rows['15-fighter_score_diff'] = fighter_score_diff_vect(test_new_rows['fighter'], test_new_rows['opponent'], test_new_rows['date'],15)


In [44]:
crapcolumns = list(ufcfightscrap.columns)
test_new_rows = test_new_rows[crapcolumns]

In [47]:
all(ufcfightscrap.columns==test_new_rows.columns)

True

In [8]:
#joining test_new_rows to ufcfightscrap to get the most updated version
frames = [test_new_rows, ufcfightscrap]
updated_ufcfightscrap = pd.concat(frames,ignore_index=True)

In [9]:
pd.set_option('display.max_columns', None)


In [10]:
updated_ufcfightscrap.iloc[0:100]

date                                          fight_url  \
0   March 26, 2022  http://ufcstats.com/fight-details/c4e866d9dd60...   
1   March 26, 2022  http://ufcstats.com/fight-details/c4e866d9dd60...   
2   March 26, 2022  http://ufcstats.com/fight-details/1b650c76387c...   
3   March 26, 2022  http://ufcstats.com/fight-details/1b650c76387c...   
4   March 26, 2022  http://ufcstats.com/fight-details/f3c37e23ca30...   
..             ...                                                ...   
95  March 05, 2022  http://ufcstats.com/fight-details/59d44b688888...   
96  March 05, 2022  http://ufcstats.com/fight-details/c420c376cc1d...   
97  March 05, 2022  http://ufcstats.com/fight-details/c420c376cc1d...   
98  March 05, 2022  http://ufcstats.com/fight-details/36d9337a27b7...   
99  March 05, 2022  http://ufcstats.com/fight-details/36d9337a27b7...   

                                            event_url result          fighter  \
0   http://ufcstats.com/event-details/1fac46d466ab...      W   Curtis Blaydes   
1   http://ufcstats.com/event-details/1fac46d466ab...      L    Chris Daukaus   
2   http://ufcstats.com/event-details/1fac46d466ab...      W     Alexa Grasso   
3   http://ufcstats.com/event-details/1fac46d466ab...      L      Joanne Wood   
4   http://ufcstats.com/event-details/1fac46d466ab...      W  Bryan Barberena   
..                                                ...    ...              ...   
95  http://ufcstats.com/event-details/75e0d3cc2262...      L   Brian Kelleher   
96  http://ufcstats.com/event-details/75e0d3cc2262...      W      Tim Elliott   
97  http://ufcstats.com/event-details/75e0d3cc2262...      L  Tagir Ulanbekov   
98  http://ufcstats.com/event-details/75e0d3cc2262...      W    Ludovit Klein   
99  http://ufcstats.com/event-details/75e0d3cc2262...      L    Devonte Smith   

             opponent           division  method round  time  \
0       Chris Daukaus        Heavyweight  KO/TKO     2  0:17   
1      Curtis Blaydes        Heavyweight  KO/TKO     2  0:17   
2         Joanne Wood  Women's Flyweight     SUB     1  3:57   
3        Alexa Grasso  Women's Flyweight     SUB     1  3:57   
4          Matt Brown       Welterweight   S-DEC     3  5:00   
..                ...                ...     ...   ...   ...   
95  Umar Nurmagomedov      Featherweight     SUB     1  3:15   
96    Tagir Ulanbekov          Flyweight   U-DEC     3  5:00   
97        Tim Elliott          Flyweight   U-DEC     3  5:00   
98      Devonte Smith        Lightweight   S-DEC     3  5:00   
99      Ludovit Klein        Lightweight   S-DEC     3  5:00   

                                          fighter_url  \
0   http://ufcstats.com/fighter-details/fa6796c55d...   
1   http://ufcstats.com/fighter-details/d0aaedcf7c...   
2   http://ufcstats.com/fighter-details/e8b731feff...   
3   http://ufcstats.com/fighter-details/12f91bfa8f...   
4   http://ufcstats.com/fighter-details/a331233f59...   
..                                                ...   
95  http://ufcstats.com/fighter-details/7be14eaed4...   
96  http://ufcstats.com/fighter-details/c96d9178c9...   
97  http://ufcstats.com/fighter-details/f00ac08ab0...   
98  http://ufcstats.com/fighter-details/5b86d491d6...   
99  http://ufcstats.com/fighter-details/207773daeb...   

                                         opponent_url knockdowns sub_attempts  \
0   http://ufcstats.com/fighter-details/d0aaedcf7c...          1            0   
1   http://ufcstats.com/fighter-details/fa6796c55d...          0            0   
2   http://ufcstats.com/fighter-details/12f91bfa8f...          0            1   
3   http://ufcstats.com/fighter-details/e8b731feff...          0            0   
4   http://ufcstats.com/fighter-details/31123249b0...          0            0   
..                                                ...        ...          ...   
95  http://ufcstats.com/fighter-details/2b6fc1c027...          0            0   
96  http://ufcstats.com/fighter-details/f00ac08ab0...         

In [11]:
#saving the updated ufcfightscrap file
updated_ufcfightscrap.to_csv('src/content/data/processed/ufc_fights_crap.csv', index = False)

In [13]:
#here is the list of all stats available (besides stance), does not include names or result
computed_statistics=[u'fighter_wins', u'fighter_losses', u'fighter_age',u'fighter_height',
                    u'fighter_reach', u'fighter_L5Y_wins',u'fighter_L5Y_losses', u'fighter_L2Y_wins', u'fighter_L2Y_losses',
                    u'fighter_ko_wins',u'fighter_ko_losses',u'fighter_L5Y_ko_wins',u'fighter_L5Y_ko_losses',u'fighter_L2Y_ko_wins',
                    u'fighter_L2Y_ko_losses',u'fighter_sub_wins',u'fighter_sub_losses',u'fighter_L5Y_sub_wins',
                    u'fighter_L5Y_sub_losses', u'fighter_L2Y_sub_wins', u'fighter_L2Y_sub_losses',
                    u'fighter_inf_knockdowns_avg',
                    u'fighter_inf_pass_avg',
                    u'fighter_inf_reversals_avg',
                    u'fighter_inf_sub_attempts_avg',
                    u'fighter_inf_takedowns_landed_avg',
                    u'fighter_inf_takedowns_attempts_avg',
                    u'fighter_inf_sig_strikes_landed_avg',
                    u'fighter_inf_sig_strikes_attempts_avg',
                    u'fighter_inf_total_strikes_landed_avg',
                    u'fighter_inf_total_strikes_attempts_avg',
                    u'fighter_inf_head_strikes_landed_avg',
                    u'fighter_inf_head_strikes_attempts_avg',
                    u'fighter_inf_body_strikes_landed_avg',
                    u'fighter_inf_body_strikes_attempts_avg',
                    u'fighter_inf_leg_strikes_landed_avg',
                    u'fighter_inf_leg_strikes_attempts_avg',
                    u'fighter_inf_distance_strikes_landed_avg',
                    u'fighter_inf_distance_strikes_attempts_avg',
                    u'fighter_inf_clinch_strikes_landed_avg',
                    u'fighter_inf_clinch_strikes_attempts_avg',
                    u'fighter_inf_ground_strikes_landed_avg',
                    u'fighter_inf_ground_strikes_attempts_avg',
                    u'fighter_abs_knockdowns_avg',
                    u'fighter_abs_pass_avg',
                    u'fighter_abs_reversals_avg',
                    u'fighter_abs_sub_attempts_avg',
                    u'fighter_abs_takedowns_landed_avg',
                    u'fighter_abs_takedowns_attempts_avg',
                    u'fighter_abs_sig_strikes_landed_avg',
                    u'fighter_abs_sig_strikes_attempts_avg',
                    u'fighter_abs_total_strikes_landed_avg',
                    u'fighter_abs_total_strikes_attempts_avg',
                    u'fighter_abs_head_strikes_landed_avg',
                    u'fighter_abs_head_strikes_attempts_avg',
                    u'fighter_abs_body_strikes_landed_avg',
                    u'fighter_abs_body_strikes_attempts_avg',
                    u'fighter_abs_leg_strikes_landed_avg',
                    u'fighter_abs_leg_strikes_attempts_avg',
                    u'fighter_abs_distance_strikes_landed_avg',
                    u'fighter_abs_distance_strikes_attempts_avg',
                    u'fighter_abs_clinch_strikes_landed_avg',
                    u'fighter_abs_clinch_strikes_attempts_avg',
                    u'fighter_abs_ground_strikes_landed_avg',
                    u'fighter_abs_ground_strikes_attempts_avg',
                    u'opponent_wins', u'opponent_losses', u'opponent_age',  u'opponent_height',
                    u'opponent_reach',   u'opponent_L5Y_wins', u'opponent_L5Y_losses', u'opponent_L2Y_wins', u'opponent_L2Y_losses', 
                    u'opponent_ko_wins', u'opponent_ko_losses', u'opponent_L5Y_ko_wins', u'opponent_L5Y_ko_losses', u'opponent_L2Y_ko_wins',
                    u'opponent_L2Y_ko_losses', u'opponent_sub_wins', u'opponent_sub_losses',u'opponent_L5Y_sub_wins', 
                    u'opponent_L5Y_sub_losses', u'opponent_L2Y_sub_wins', u'opponent_L2Y_sub_losses',
                    u'opponent_inf_knockdowns_avg',
                    u'opponent_inf_pass_avg',
                    u'opponent_inf_reversals_avg',
                    u'opponent_inf_sub_attempts_avg',
                    u'opponent_inf_takedowns_landed_avg',
                    u'opponent_inf_takedowns_attempts_avg',
                    u'opponent_inf_sig_strikes_landed_avg',
                    u'opponent_inf_sig_strikes_attempts_avg',
                    u'opponent_inf_total_strikes_landed_avg',
                    u'opponent_inf_total_strikes_attempts_avg',
                    u'opponent_inf_head_strikes_landed_avg',
                    u'opponent_inf_head_strikes_attempts_avg',
                    u'opponent_inf_body_strikes_landed_avg',
                    u'opponent_inf_body_strikes_attempts_avg',
                    u'opponent_inf_leg_strikes_landed_avg',
                    u'opponent_inf_leg_strikes_attempts_avg',
                    u'opponent_inf_distance_strikes_landed_avg',
                    u'opponent_inf_distance_strikes_attempts_avg',
                    u'opponent_inf_clinch_strikes_landed_avg',
                    u'opponent_inf_clinch_strikes_attempts_avg',
                    u'opponent_inf_ground_strikes_landed_avg',
                    u'opponent_inf_ground_strikes_attempts_avg',
                    u'opponent_abs_knockdowns_avg',
                    u'opponent_abs_pass_avg',
                    u'opponent_abs_reversals_avg',
                    u'opponent_abs_sub_attempts_avg',
                    u'opponent_abs_takedowns_landed_avg',
                    u'opponent_abs_takedowns_attempts_avg',
                    u'opponent_abs_sig_strikes_landed_avg',
                    u'opponent_abs_sig_strikes_attempts_avg',
                    u'opponent_abs_total_strikes_landed_avg',
                    u'opponent_abs_total_strikes_attempts_avg',
                    u'opponent_abs_head_strikes_landed_avg',
                    u'opponent_abs_head_strikes_attempts_avg',
                    u'opponent_abs_body_strikes_landed_avg',
                    u'opponent_abs_body_strikes_attempts_avg',
                    u'opponent_abs_leg_strikes_landed_avg',
                    u'opponent_abs_leg_strikes_attempts_avg',
                    u'opponent_abs_distance_strikes_landed_avg',
                    u'opponent_abs_distance_strikes_attempts_avg',
                    u'opponent_abs_clinch_strikes_landed_avg',
                    u'opponent_abs_clinch_strikes_attempts_avg',
                    u'opponent_abs_ground_strikes_landed_avg',
                    u'opponent_abs_ground_strikes_attempts_avg',
                    u'fighter_stance',
                    u'opponent_stance',
                '1-fight_math',
                '6-fight_math',
                '4-fighter_score_diff',
                '9-fighter_score_diff',
                 '15-fighter_score_diff',]



In [14]:
#list containing all columns of any interest
relevant_list=['date','division','fighter','opponent','result','method']
relevant_list.extend(computed_statistics)

In [15]:
#creates a clean file with only columns which are relevant to predicting
updated_ufc_fights=updated_ufcfightscrap[relevant_list]

In [16]:
#lets randomly remove one of every two copied fights
random_indices=[]
for i in range(0,len(updated_ufc_fights['fighter_wins']),2):
    random_indices.append(random.choice([i,i+1]))
    
updated_ufc_fights=updated_ufc_fights.drop(random_indices)

In [17]:
updated_ufc_fights

date           division          fighter        opponent  \
1         March 26, 2022        Heavyweight    Chris Daukaus  Curtis Blaydes   
3         March 26, 2022  Women's Flyweight      Joanne Wood    Alexa Grasso   
4         March 26, 2022       Welterweight  Bryan Barberena      Matt Brown   
6         March 26, 2022          Flyweight  Kai Kara-France   Askar Askarov   
9         March 26, 2022       Welterweight      Max Griffin      Neil Magny   
...                  ...                ...              ...             ...   
13061  November 12, 1993        Open Weight     Kevin Rosier  Gerard Gordeau   
13062  November 12, 1993        Open Weight     Ken Shamrock   Patrick Smith   
13065  November 12, 1993        Open Weight    Art Jimmerson    Royce Gracie   
13066  November 12, 1993        Open Weight     Kevin Rosier    Zane Frazier   
13068  November 12, 1993        Open Weight   Gerard Gordeau      Teila Tuli   

      result  method  fighter_wins  fighter_losses fighter_age  \
1          L  KO/TKO           4.0             1.0        32.0   
3          L     SUB           1.0             2.0         0.0   
4          W   S-DEC           7.0             6.0        32.0   
6          W   U-DEC           6.0             2.0        29.0   
9          L   S-DEC           6.0             6.0        36.0   
...      ...     ...           ...             ...         ...   
13061      L  KO/TKO           0.0             0.0     unknown   
13062      W     SUB           0.0             0.0          29   
13065      L     SUB           0.0             0.0          30   
13066      W  KO/TKO           0.0             0.0     unknown   
13068      W  KO/TKO           0.0             0.0          34   

           fighter_height fighter_reach  fighter_L5Y_wins  fighter_L5Y_losses  \
1                   190.5         190.5               4.0                 1.0   
3                 unknown       unknown               1.0                 2.0   
4                  182.88        182.88               4.0                 4.0   
6                  162.56        162.56               6.0                 2.0   
9                  180.34        180.34               5.0                 5.0   
...                   ...           ...               ...                 ...   
13061              193.04       unknown               0.0                 0.0   
13062              185.42        182.88               0.0                 0.0   
13065              185.42       unknown               0.0                 0.0   
13066              193.04       unknown               0.0                 0.0   
13068  195.57999999999998       unknown               0.0                 0.0   

       fighter_L2Y_wins  fighter_L2Y_losses  fighter_ko_wins  \
1                   4.0                 1.0              4.0   
3                   1.0                 2.0              0.0   
4                   2.0                 1.0              3.0   
6                   2.0                 1.0              2.0   
9                   3.0                 0.0              3.0   
...                 ...                 ...              ...   
13061               0.0                 0.0              0.0   
13062               0.0                 0.0              0.0   
13065               0.0                 0.0              0.0   
13066               0.0                 0.0              0.0   
13068               0.0                 0.0              0.0   

       fighter_ko_losses  fighter_L5Y_ko_wins  fighter_L5Y_ko_losses  \
1                    1.0                  4.0                    1.0   
3                    0.0                  0.0                    0.0   
4                    2.0                  2.0                    2.0   
6                    0.0                  2.0                    0.0   
9                    1.0                  2.0                    0.0   
...                  ...                  ...                    ...   
13061               

In [18]:
#we worked hard to build this, lets save it (only run this once we're sure that the new file is correct)
updated_ufc_fights.to_csv('src/content/data/processed/ufc_fights.csv', index = False)

In [19]:
ufcfightscrap.columns

Index(['date', 'fight_url', 'event_url', 'result', 'fighter', 'opponent',
       'division', 'method', 'round', 'time',
       ...
       'opponent_abs_clinch_strikes_attempts_avg',
       'opponent_abs_ground_strikes_landed_avg',
       'opponent_abs_ground_strikes_attempts_avg', 'fighter_stance',
       'opponent_stance', '1-fight_math', '6-fight_math',
       '4-fighter_score_diff', '9-fighter_score_diff',
       '15-fighter_score_diff'],
      dtype='object', length=172)

In [20]:
df1 = pd.read_json('src/content/data/external/ufc_fights.json')
df1

ValueError: Expected object or value

In [ ]:
#ufcfighterscrap=pd.read_csv('fighter_stats.csv',sep=',')
ufcfighterscrap.columns

In [ ]:
#convert fighter_stats.csv to json files to read via javascript in website
csvFilePath = r'src/content/data/processed/fighter_stats.csv'
jsonFilePath = r'src/content/data/external/fighter_stats.json'
 
# Call the make_json function with 'name' column as main key
make_json(csvFilePath, jsonFilePath, 'name')

In [ ]:
df2 = pd.read_json('src/content/data/external/fighter_stats.json')
df2

In [24]:
updated_ufcfightscrap['index'] = updated_ufcfightscrap['fighter']

for i in range(len(updated_ufcfightscrap['date'])):
    updated_ufcfightscrap['index'][i]=i

/var/folders/zr/jttn57m976g1m1_y2p2t8zwm0000gn/T/ipykernel_12390/2352096194.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  updated_ufcfightscrap['index'][i]=i


In [25]:
json_columns = ['date',
'result',
'fighter',
'opponent',
'division',
'method',
'round',
'time',
'knockdowns',
'sub_attempts',
'pass',
'reversals',
'takedowns_landed',
'takedowns_attempts',
'sig_strikes_landed',
'sig_strikes_attempts',
'total_strikes_landed',
'total_strikes_attempts',
'head_strikes_landed',
'head_strikes_attempts',
'body_strikes_landed',
'body_strikes_attempts',
'leg_strikes_landed',
'leg_strikes_attempts',
'distance_strikes_landed',
'distance_strikes_attempts',
'clinch_strikes_landed',
'clinch_strikes_attempts',
'ground_strikes_landed',
'ground_strikes_attempts',
'fighter_stance',
'opponent_stance',
'index',]

In [26]:
ufcfightscrap_for_json = updated_ufcfightscrap[json_columns]

In [27]:
ufcfightscrap_for_json

date result          fighter        opponent  \
0         March 26, 2022      W   Curtis Blaydes   Chris Daukaus   
1         March 26, 2022      L    Chris Daukaus  Curtis Blaydes   
2         March 26, 2022      W     Alexa Grasso     Joanne Wood   
3         March 26, 2022      L      Joanne Wood    Alexa Grasso   
4         March 26, 2022      W  Bryan Barberena      Matt Brown   
...                  ...    ...              ...             ...   
13065  November 12, 1993      L    Art Jimmerson    Royce Gracie   
13066  November 12, 1993      W     Kevin Rosier    Zane Frazier   
13067  November 12, 1993      L     Zane Frazier    Kevin Rosier   
13068  November 12, 1993      W   Gerard Gordeau      Teila Tuli   
13069  November 12, 1993      L       Teila Tuli  Gerard Gordeau   

                division  method round  time knockdowns sub_attempts pass  \
0            Heavyweight  KO/TKO     2  0:17          1            0  NaN   
1            Heavyweight  KO/TKO     2  0:17          0            0  NaN   
2      Women's Flyweight     SUB     1  3:57          0            1  NaN   
3      Women's Flyweight     SUB     1  3:57          0            0  NaN   
4           Welterweight   S-DEC     3  5:00          0            0  NaN   
...                  ...     ...   ...   ...        ...          ...  ...   
13065        Open Weight     SUB     1  2:18          0            0  0.0   
13066        Open Weight  KO/TKO     1  4:20          2            0  0.0   
13067        Open Weight  KO/TKO     1  4:20          0            0  0.0   
13068        Open Weight  KO/TKO     1  0:26          0            0  0.0   
13069        Open Weight  KO/TKO     1  0:26          0            0  0.0   

      reversals takedowns_landed takedowns_attempts sig_strikes_landed  \
0             0                0                  0                 30   
1             0                0                  0                 13   
2             0                2                  2                 16   
3             0                0                  0                 34   
4             1                0                  0                111   
...         ...              ...                ...                ...   
13065         0                0                  0                  0   
13066         0                0                  0                 15   
13067         0                0                  0                 12   
13068         0                0                  0                  3   
13069         0                0                  1                  0   

      sig_strikes_attempts total_strikes_landed total_strikes_attempts  \
0                       63                   30                     63   
1                       38                   13                     38   
2                       28                   50                     65   
3                       49                   61                     76   
4                      211                  186                    289   
...                    ...                  ...                    ...   
13065                    0                    0                      0   
13066                   27                   38                     53   
13067                   28                   13                     29   
13068                    5                    3                      5   
13069                    1                    0                      1   

      head_strikes_landed head_strikes_attempts body_strikes_landed  \
0                      23                    55                   1   
1                      12                    36                   1   
2                      13                    25                   0   
3                      22                    33                   4   
4                      82                   170                  12   
...                   ...                   ...                 ...   
130

In [28]:
#make new csv just to send it to json
#this is inefficient and wastes space... but its just because its the only way I know to make a json file
#of the correct format (fix needed but not super important)
ufcfightscrap_for_json.to_csv('src/content/data/processed/ufcfightscrap.csv', index = False)

In [29]:
#convert fighter_stats.csv to json files to read via javascript in website
csvFilePath = r'src/content/data/processed/ufcfightscrap.csv'
jsonFilePath = r'src/content/data/external/ufcfightscrap.json'
 
# Call the make_json function with 'fight_url' column as main key
make_json(csvFilePath, jsonFilePath, 'index')

In [ ]:
#add code here to automatically send these to github

In [ ]:
#So the only files we really need are fight_hist,fighter_stats,ufc_fights,ufc_fights_crap, and ufc_fighters